In [16]:
from surprise import KNNBasic
import os
from surprise import SVD
from surprise import Dataset
from surprise import evaluate, print_perf
import numpy as np
import pandas as pd
import operator
import surprise

In [2]:
#optional
from surprise import Reader
# path to dataset file
file_path = os.path.expanduser('/Users/LukeDuan/Desktop/CSC478/ml-latest-small/ratings.csv')

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating timestamp', sep=',')

data = Dataset.load_from_file(file_path, reader=reader)
data.split(n_folds=5)

In [10]:
#optional
def existingUserOld(path,algo,uid,movie):
   
    # path to dataset file
    file_path = os.path.expanduser(path)

    # As we're loading a custom dataset, we need to define a reader. In the
    # movielens-100k dataset, each line has the following format:
    # 'user item rating timestamp', separated by '\t' characters.
    reader = Reader(line_format='user item rating timestamp', sep=',')

    data = Dataset.load_from_file(file_path, reader=reader)
    # Build an algorithm, and train it.
    trainset = data.build_full_trainset()
    algo.train(trainset)
    rating={}
    movieId=movie.movieId
    movieId=list(movieId)
    for i in movieId:
        pred = algo.predict(uid, str(i), r_ui=5, verbose=True)
        rating.update({i: pred.est})
    sorted_rating = sorted(rating.items(), key=operator.itemgetter(1))
    sorted_rating.reverse()
    watch=pd.DataFrame(data.raw_ratings,columns = ['u','i','r','t'])
    watched=watch[watch['u'] == uid]
    for j in range(0,20):
        for i in watched['i']:
            if sorted_rating[j][0]==i:
                sorted_rating.pop(j)
    return sorted_rating
   

In [6]:
#become the movie dataset from SQLite
movie=pd.read_csv('/Users/LukeDuan/Desktop/CSC478/ml-latest-small/movies.csv')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
#results
sorted_rating=existingUserOld('/Users/LukeDuan/Desktop/CSC478/ml-latest-small/ratings.csv',SVD(),str(608),movie)

user: 608        item: 1          r_ui = 5.00   est = 4.07   {u'was_impossible': False}
user: 608        item: 2          r_ui = 5.00   est = 3.57   {u'was_impossible': False}
user: 608        item: 3          r_ui = 5.00   est = 3.26   {u'was_impossible': False}
user: 608        item: 4          r_ui = 5.00   est = 2.63   {u'was_impossible': False}
user: 608        item: 5          r_ui = 5.00   est = 3.31   {u'was_impossible': False}
user: 608        item: 6          r_ui = 5.00   est = 4.01   {u'was_impossible': False}
user: 608        item: 7          r_ui = 5.00   est = 3.47   {u'was_impossible': False}
user: 608        item: 8          r_ui = 5.00   est = 3.73   {u'was_impossible': False}
user: 608        item: 9          r_ui = 5.00   est = 3.15   {u'was_impossible': False}
user: 608        item: 10         r_ui = 5.00   est = 3.54   {u'was_impossible': False}
user: 608        item: 11         r_ui = 5.00   est = 3.84   {u'was_impossible': False}
user: 608        item: 12       

In [14]:
sorted_rating[0:5]

[(926, 4.7786458138866372),
 (905, 4.7565954149902847),
 (3462, 4.7557946581498776),
 (969, 4.7240426534129378),
 (7502, 4.7188883987829096)]

In [28]:
#creating a new dataset object
from surprise import dataset
class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, df, reader):

        self.raw_ratings = [(uid, iid, r, timestamp) for (uid, iid, r, timestamp) in
                            zip(df['u'], df['i'], df['r'],df['t'])]
        self.reader=reader

In [29]:
#entry will be a pd dataframe from web
#path is the place will the training set is stored
#Function to add new lines of rating entry to the training set
def addEntrytoTrain(path,entry):
    old=pd.read_csv(path,header=None)
    newmovie = old.append(pd.DataFrame(data = entry), ignore_index=True)
    newmovie.columns=['u','i','r','t']
    reader = Reader(line_format='user item rating timestamp', rating_scale=(1, 5))
    data=MyDataset(newmovie,reader)
    return data



In [26]:
entry=pd.read_csv('/Users/LukeDuan/Desktop/CSC478/ml-latest-small/newrating.csv',header=None)
entry

,0,1,2,3
0,672,1061,3.0,1260759144
1,672,1172,4.0,1260759205
2,672,1263,2.0,1260759151
3,672,1287,2.0,1260759187
4,672,1293,2.0,1260759148
5,672,1339,3.5,1260759125
6,672,1343,2.0,1260759131


In [44]:
datanew=addEntrytoTrain('/Users/LukeDuan/Desktop/CSC478/ml-latest-small/ratings.csv',entry)

In [57]:
datanew.raw_ratings[0][0]

1

In [58]:
##The function to predict and come up with the top 20 movies
def newUserRating(data,newuid,algo,movie):
    # Build an algorithm, and train it.
    trainset = data.build_full_trainset()
    algo.train(trainset)
    rating={}
    movieId=movie.movieId
    movieId=list(movieId)
    for i in movieId:
        pred = algo.predict(newuid, i, r_ui=5, verbose=True)
        rating.update({i: pred.est})
    sorted_rating = sorted(rating.items(), key=operator.itemgetter(1))
    sorted_rating.reverse()
    watch=pd.DataFrame(data.raw_ratings,columns = ['u','i','r','t'])
    watched=watch[watch['u'] == newuid]
    for j in range(0,20):
        for i in watched['i']:
            if sorted_rating[j][0]==i:
                sorted_rating.pop(j)
    return sorted_rating

In [60]:
sortednewrating=newUserRating(datanew,672,SVD(),movie)

user: 672        item: 1          r_ui = 5.00   est = 3.29   {u'was_impossible': False}
user: 672        item: 2          r_ui = 5.00   est = 2.82   {u'was_impossible': False}
user: 672        item: 3          r_ui = 5.00   est = 2.52   {u'was_impossible': False}
user: 672        item: 4          r_ui = 5.00   est = 2.00   {u'was_impossible': False}
user: 672        item: 5          r_ui = 5.00   est = 2.62   {u'was_impossible': False}
user: 672        item: 6          r_ui = 5.00   est = 3.25   {u'was_impossible': False}
user: 672        item: 7          r_ui = 5.00   est = 2.70   {u'was_impossible': False}
user: 672        item: 8          r_ui = 5.00   est = 2.88   {u'was_impossible': False}
user: 672        item: 9          r_ui = 5.00   est = 2.44   {u'was_impossible': False}
user: 672        item: 10         r_ui = 5.00   est = 2.89   {u'was_impossible': False}
user: 672        item: 11         r_ui = 5.00   est = 3.07   {u'was_impossible': False}
user: 672        item: 12       

In [61]:
sortednewrating

[(858, 3.955328463817716),
 (318, 3.8909699092991636),
 (50, 3.7918019687601694),
 (296, 3.7907191820241071),
 (1221, 3.7703755091542734),
 (905, 3.761426445624565),
 (969, 3.7571878414859636),
 (926, 3.7568877548971287),
 (7502, 3.7370977392049096),
 (608, 3.727344385031488),
 (3462, 3.7153107913746104),
 (527, 3.7116329427187731),
 (904, 3.7108644274890117),
 (593, 3.7067275870845142),
 (6016, 3.7057271055077203),
 (913, 3.705279739472092),
 (899, 3.7041231950371234),
 (912, 3.7021711107663378),
 (1203, 3.6969166537413818),
 (1945, 3.6913107836476109),
 (2064, 3.6843737791613025),
 (1196, 3.6832777716825644),
 (6787, 3.6822476756033109),
 (2959, 3.6794189006661417),
 (8132, 3.6777336132386473),
 (4226, 3.6725300899069682),
 (908, 3.672166819720001),
 (1252, 3.6720279444300696),
 (260, 3.6719443559218412),
 (1228, 3.6646291733945731),
 (1172, 3.6642073460246105),
 (2571, 3.6635272533927168),
 (2318, 3.6620666423667996),
 (923, 3.6546596638293218),
 (48516, 3.6500583530144377),
 (1193,